# 🎓 Interactive NCERT Question Generator & Evaluator

## What This Notebook Does

This interactive system:
1. 📚 **Extracts content** from your NCERT-10-Science.pdf textbook
2. 🎯 **Generates questions** from chapters you select
3. ✍️ **Accepts your answers** (type directly or upload PDF)
4. 🔍 **Evaluates** your answers using AI
5. 📊 **Shows which topics** you need to focus on

---

## Step 1: Setup - Import Libraries

In [ ]:
import sys
sys.path.append('src')

from ncert_question_generator import NCERTQuestionGenerator, InteractiveExamSystem
from feedback_generator import FeedbackGenerator
import json
from pathlib import Path

print("✓ Libraries imported successfully!")

## Step 2: Initialize System with NCERT Textbook

In [ ]:
# Initialize the interactive exam system
ncert_pdf = "NCERT-10-Science.pdf"
exam_system = InteractiveExamSystem(ncert_pdf)

print("📚 Setting up system...")
if exam_system.setup():
    print("✓ System ready!")
else:
    print("✗ Error loading NCERT textbook")

## Step 3: View Available Chapters

In [ ]:
print("📖 Available Chapters in NCERT Class 10 Science:\n")
print("="*70)

for ch_num in sorted(exam_system.generator.chapters.keys()):
    chapter = exam_system.generator.chapters[ch_num]
    print(f"Chapter {ch_num}: {chapter['title']}")

print("="*70)

## Step 4: SELECT - How Many Questions Do You Want?

**Customize your exam below:**

In [ ]:
# ========== CUSTOMIZE HERE ==========

# Which chapters do you want questions from? (list of chapter numbers)
selected_chapters = [1, 2, 3]  # Change this list!

# How many questions per chapter?
questions_per_chapter = 2  # Change this number!

# ====================================

# Generate questions
questions = exam_system.generate_exam(selected_chapters, questions_per_chapter)

print(f"\n✓ Generated {len(questions)} questions!")
print(f"  Chapters: {selected_chapters}")
print(f"  Questions per chapter: {questions_per_chapter}")

## Step 5: View Your Exam Questions

In [ ]:
# Display the generated questions
exam_system.display_questions()

## Step 6: Write Your Answers

**Option A: Type your answers directly below**

**Option B: Upload PDF answers (see next cell)**

In [ ]:
# ========== WRITE YOUR ANSWERS HERE ==========

# Answer for Question 1
answer_1 = """
Write your answer for question 1 here...

You can write multiple paragraphs.
Include all the concepts you know.
"""

# Answer for Question 2
answer_2 = """
Write your answer for question 2 here...
"""

# Answer for Question 3 (add more as needed)
answer_3 = """
Write your answer for question 3 here...
"""

# Collect all answers
student_answers = [answer_1, answer_2, answer_3]

# Submit answers
for i, answer in enumerate(student_answers[:len(questions)]):
    q_id = questions[i]['id']
    exam_system.accept_answer(q_id, answer_text=answer)

print(f"✓ Submitted {len(student_answers)} answers!")

### Alternative: Upload PDF Answers

In [ ]:
# If you have answers in PDF format, use this instead:

# Example:
# exam_system.accept_answer(questions[0]['id'], answer_pdf="my_answer_q1.pdf")
# exam_system.accept_answer(questions[1]['id'], answer_pdf="my_answer_q2.pdf")

print("PDF upload option available - uncomment and modify the code above")

## Step 7: EVALUATE - Get Your Marks!

In [ ]:
# Evaluate all answers
results = exam_system.evaluate_exam()

print("\n" + "="*70)
print("📊 EVALUATION RESULTS")
print("="*70)

total_marks = 0
total_max = 0

for i, result in enumerate(results, 1):
    print(f"\nQuestion {i}: {result['question_text'][:50]}...")
    print(f"Chapter: {result['chapter']}")
    print(f"Marks: {result['marks_obtained']}/{result['max_marks']} ({result['percentage']:.1f}%)")
    print("-" * 70)
    
    total_marks += result['marks_obtained']
    total_max += result['max_marks']

overall_percentage = (total_marks / total_max * 100) if total_max > 0 else 0

print("\n" + "="*70)
print(f"📈 OVERALL SCORE: {total_marks:.1f}/{total_max} ({overall_percentage:.1f}%)")
print("="*70)

## Step 8: Detailed Feedback for Each Answer

In [ ]:
feedback_gen = FeedbackGenerator()

for i, result in enumerate(results, 1):
    print(f"\n{'='*70}")
    print(f"FEEDBACK FOR QUESTION {i}")
    print(f"{'='*70}")
    
    feedback = feedback_gen.generate_feedback(result, verbose=False)
    feedback_text = feedback_gen.format_feedback_text(feedback)
    print(feedback_text)

## Step 9: 🎯 TOPIC-WISE PERFORMANCE - What to Focus On?

In [ ]:
# Generate topic-wise performance analysis
topic_feedback, performance = exam_system.generate_topic_feedback(results)

print("\n" + "="*70)
print("📚 TOPIC-WISE PERFORMANCE ANALYSIS")
print("="*70)

print("\nYour Performance by Chapter:\n")

for chapter, data in sorted(topic_feedback.items()):
    status_emoji = "✅" if data['status'] == 'Strong' else "⚠️" if data['status'] == 'Good' else "❌"
    print(f"{status_emoji} {chapter}")
    print(f"   Average Score: {data['average_score']:.1f}%")
    print(f"   Status: {data['status']}")
    print(f"   Questions Attempted: {data['questions_attempted']}")
    print()

print("="*70)
print("\n📌 FOCUS AREAS:\n")

weak_topics = [ch for ch, data in topic_feedback.items() if data['status'] == 'Needs Focus']

if weak_topics:
    print("You should focus more on these chapters:")
    for topic in weak_topics:
        print(f"  • {topic} ({topic_feedback[topic]['average_score']:.1f}%)")
else:
    print("✓ Great job! No major weak areas detected.")
    print("  Keep practicing to maintain your performance!")

print("\n" + "="*70)

## Step 10: Save Your Results

In [ ]:
# Save detailed results to JSON
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Prepare summary
exam_summary = {
    "total_questions": len(results),
    "total_marks_obtained": round(total_marks, 2),
    "total_max_marks": total_max,
    "overall_percentage": round(overall_percentage, 2),
    "topic_performance": topic_feedback,
    "question_results": [
        {
            "question": r['question_text'],
            "chapter": r['chapter'],
            "marks": round(r['marks_obtained'], 2),
            "max_marks": r['max_marks'],
            "percentage": round(r['percentage'], 1)
        }
        for r in results
    ]
}

# Save to file
output_file = output_dir / "ncert_exam_results.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(exam_summary, f, indent=2, ensure_ascii=False)

print(f"✓ Results saved to: {output_file}")
print(f"\nYou can review your detailed results anytime by opening this file!")

## 🎉 Summary

### What You Did:
1. ✅ Loaded NCERT Class 10 Science textbook
2. ✅ Selected chapters and number of questions
3. ✅ Generated custom questions from NCERT content
4. ✅ Wrote/uploaded your answers
5. ✅ Got AI-powered evaluation with detailed feedback
6. ✅ Identified topics that need more focus

### Next Steps:
- 📖 Review weak topics from the performance analysis
- 🔄 Run this notebook again with different chapters
- 📈 Track your progress over time
- 🎯 Focus on chapters marked "Needs Focus"

---

**Keep practicing and improving! 🚀**